In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Chargement des données
X_path = "C:\\Users\\user\\OneDrive\\DATASIENCETEST\\PROJET\\Data\\Update\\X_clean_before_modeling.csv"
y_path = "C:\\Users\\user\\OneDrive\\DATASIENCETEST\\PROJET\\Data\\Update\\Y_train_CVw08PX.csv"

X = pd.read_csv(X_path)
y = pd.read_csv(y_path, index_col=0).squeeze().map(str)

In [6]:
# Vérification de l'existence des colonnes avant de les supprimer pour éviter des erreurs
colonnes_a_supprimer = ['designation', 'description', 'text_combined']
for colonne in colonnes_a_supprimer:
    if colonne in X.columns:
        X.drop(colonne, axis=1, inplace=True)

In [7]:
# Vérifier si des valeurs NaN existent dans le DataFrame
nan_exists = X.isna().any().any()
print(f"Des valeurs NaN existent-elles dans le DataFrame ? {nan_exists}")


Des valeurs NaN existent-elles dans le DataFrame ? True


In [8]:
# Identifier les colonnes qui contiennent des valeurs NaN
nan_columns = X.isna().any()
print("Colonnes contenant des valeurs NaN :")
print(nan_columns[nan_columns == True])


Colonnes contenant des valeurs NaN :
text_clean    True
dtype: bool


In [9]:
# Compter le nombre de valeurs NaN dans chaque colonne
nan_counts = X.isna().sum()
print("Nombre de valeurs NaN par colonne :")
print(nan_counts)


Nombre de valeurs NaN par colonne :
productid     0
imageid       0
text_clean    3
dtype: int64


In [10]:
# Remplacer les NaN par des chaînes vides dans tout le Df
X.fillna('', inplace=True)

# Vérification de nouveau pour s'assurer que les NaN ont été correctement remplacés
nan_exists_after = X.isna().any().any()
print(f"Des valeurs NaN existent-elles dans le DataFrame après le remplacement ? {nan_exists_after}")


Des valeurs NaN existent-elles dans le DataFrame après le remplacement ? False


In [11]:
# Ajout d'une feature simple basée sur la longueur du texte
X['text_length'] = X['text_clean'].apply(len)

In [12]:
# Séparation des features textuelles et numériques
X_text = X['text_clean']
X_num = X[['text_length']]

In [13]:
# Division des données
X_train_text, X_test_text, X_train_num, X_test_num, y_train, y_test = train_test_split(X_text, X_num, y, test_size=0.2, random_state=42)

### Rééquilibrage et Vectorisation avec Optimisation des Hyperparamètres

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.over_sampling import SMOTE
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X['text_clean'], y, test_size=0.2, random_state=42)

MultinomialNB

In [21]:
# Création du pipeline avec vectorisation et modèle
pipeline = make_pipeline_imb(
    TfidfVectorizer(stop_words=stopwords.words('french')),
    SMOTE(random_state=42),
    MultinomialNB()
)

# Définition de param_grid sans utiliser de préfixe incorrect
param_grid = {
    'tfidfvectorizer__max_features': [5000, 10000],
    'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)],
    'multinomialnb__alpha': [0.1, 1.0]
}

In [22]:
# Configuration de GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train) 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidfvectorizer',
                                        TfidfVectorizer(stop_words=['au', 'aux',
                                                                    'avec',
                                                                    'ce', 'ces',
                                                                    'dans',
                                                                    'de', 'des',
                                                                    'du',
                                                                    'elle',
                                                                    'en', 'et',
                                                                    'eux', 'il',
                                                                    'ils', 'je',
                                                                    'la', 'le',
                                                                    'les',
                                                                    'leur',
                                                                    'lui', 'ma',
                                                                    'mais',
                                                                    'me',
                                                                    'même',
                                                                    'mes',
                                                                    'moi',
                                                                    'mon', 'ne',
                                                                    'nos', ...])),
                                       ('smote', SMOTE(random_state=42)),
                                       ('multinomialnb', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'multinomialnb__alpha': [0.1, 1.0],
                         'tfidfvectorizer__max_features': [5000, 10000],
                         'tfidfvectorizer__ngram_range': [(1, 1), (1, 2)]},
             scoring='accuracy')

In [23]:
# Meilleurs paramètres et score
print("Meilleurs paramètres: ", grid_search.best_params_)
print("Meilleur score: ", grid_search.best_score_)

Meilleurs paramètres:  {'multinomialnb__alpha': 0.1, 'tfidfvectorizer__max_features': 10000, 'tfidfvectorizer__ngram_range': (1, 1)}
Meilleur score:  0.7468056353301666


In [24]:
# Prédiction sur l'ensemble de test avec le meilleur modèle
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test_text)

In [25]:
# Évaluation des performances du modèle
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

          10       0.49      0.41      0.44       612
        1140       0.63      0.82      0.71       539
        1160       0.91      0.88      0.89       786
        1180       0.45      0.60      0.52       146
        1280       0.64      0.47      0.54       961
        1281       0.45      0.44      0.45       424
        1300       0.79      0.89      0.84       974
        1301       0.88      0.90      0.89       169
        1302       0.71      0.73      0.72       507
        1320       0.79      0.66      0.72       672
        1560       0.76      0.73      0.74      1013
        1920       0.87      0.87      0.87       841
        1940       0.68      0.93      0.78       137
        2060       0.71      0.74      0.72      1029
        2220       0.69      0.86      0.76       170
        2280       0.69      0.65      0.67       942
        2403       0.74      0.64      0.69       986
        2462       0.62    